In [16]:
##### 同時推定コード #####
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

start_time = time.time()
print('start!')

####### reading data #######
df_quater = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0815_kai.csv')

df_link_odds = pd.read_csv("/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/shibuya_link_1112/shibuya_link_post_odds.csv")
l = len(df_link_odds)

df_link_integrated = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/shibuya_link_post_integrated?.csv')
L = len(df_link_integrated)

search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/user_stock_kai0015_end21/groups/group_2"
file_list = list(os.listdir(search_folder))

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/shibuya_node_corrected.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

###### 各linkのoとdの座標を入れておく配列 ###### # 高さ方向の検出を排除（床の遮蔽とか考え出すとやばいので）
oddslink_loc_array = [] 
for i in range(len(df_link_odds)):
    linkid = df_link_odds.loc[i, 'linkid']
    O = df_link_odds.loc[i, 'O']
    D = df_link_odds.loc[i, 'D']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] * 1000
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] * 1000 # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### ビーコンとリンク線分の最短距離返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    
    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    
    # 垂直ベクトルのノルムを求める
    distance = np.linalg.norm(v)
    
    # 線分の範囲内であるかチェック
    dot_product = np.dot(v1, segment)
    if dot_product < 0:
        return np.linalg.norm(x - p1)
    elif dot_product > np.dot(segment, segment):
        return np.linalg.norm(x - p2)
    else:
        return distance

###### d_arrayの用意 ###### # 各リンク(奇数側)とbleビーコンとの距離配列
d_array = np.zeros((len(df_link_odds), len(df_ble)))

for i in range(len(df_link_odds)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']*1000 # zの値をめちゃ大きくすることで，階を挟んだ電波捕捉を捨象（階の厳密な高さが不明なのと，床板・天井版による電波遮蔽を考慮するため）．階が同じなら0になるので変な影響はない
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink 2i+1, ビーコンjの距離

###### 各ビーコンからxm圏内のリンクリスト ######
linklist_near_j = []
for j in range(len(df_ble)):
    column_j = d_array[:, j]
    link_list = np.where(column_j <= 10)[0] # 10m以下の場所の指定（インデックスが得られる）  # ここは変えてみる価値あり．12.5m, 15m, 20mにしてみる
    link_list = [idx*2 + 1 for idx in link_list] # link_listの各要素はindexなので，2掛けて1を足せばlinkidになる

    linklist_near_j.append(link_list)

###### ノード間距離の配列 ######
d_node_array = np.zeros((len(df_node), len(df_node))) # d_node_arrayのi行j列がnode i+1とnode j+1の間の距離となるように計算
for i in range(len(df_node)):
    xi = df_node.loc[i, 'x']
    yi = df_node.loc[i, 'y']
    zi = df_node.loc[i, 'floor']
    i_loc = np.array([xi, yi, zi])
    
    for j in range(len(df_node)):
        xj = df_node.loc[j, 'x']
        yj = df_node.loc[j, 'y']
        zj = df_node.loc[j, 'floor']
        j_loc = np.array([xj, yj, zj])

        d_node_array[i, j] = np.linalg.norm(i_loc - j_loc)

###### ビーコンとノード間の距離の配列 ###### # ビーコンi+1とnode j+1間の距離
d_b_node_array = np.zeros((len(df_ble), len(df_node)))
for i in range(len(df_ble)):
    xb = df_ble.loc[i, 'x']
    yb = df_ble.loc[i, 'y']
    zb = df_ble.loc[i, 'floor']
    b_loc = np.array([xb, yb, zb])

    for j in range(len(df_node)):
        xj = df_node.loc[j, 'x']
        yj = df_node.loc[j, 'y']
        zj = df_node.loc[j, 'floor']
        j_loc = np.array([xj, yj, zj])

        d_b_node_array[i, j] = np.linalg.norm(b_loc - j_loc)

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
### 20秒にするのでここやり直す．
dens_jt = np.zeros((23, 60), dtype = int)
max_jt = np.zeros((23, 60), dtype = int)
mean_jt = np.zeros((23, 60), dtype = int)
#count_jt = np.zeros((23, 60), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    # beacon_idとtimestepを得る
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列 ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n)
    for i in range(n):
        O = link_data.loc[i, 'O'] # 当該linkのOnode
        D = link_data.loc[i, 'D'] # Dnode
        for j in range(n):
            if ((link_data.loc[j, 'O'] == O) or (link_data.loc[j, 'O'] == D)) or (link_data.loc[j, 'D'] == O) or (link_data.loc[j, 'D'] == D): 
                I[i, j] = 1
    return(I)

####### リンク接続行列 ####### 
def I_withd(link_data):
    n = len(link_data)
    I = np.eye(n)
    for i in range(n):
        D = link_data.loc[i, 'D']
        for j in range(n):
            if link_data.loc[j, 'O'] == D: 
                I[i, j] = 1
    return(I)

###### 接続行列 ###### 
Ind = I_nod(df_link_odds) 

###### 接続行列 ###### 吸収リンクから吸収リンクへの接続は0に直す
Id = I_withd(df_link_integrated) 
Id[64, 64] = 0

staire_link = df_link_odds[df_link_odds['staire'] == 1]['linkid'].to_list() 

###### 改札の吸収リンク
ddata = [65] # 中央の吸収リンクのみにした．NWの設定とセットで変える
# pre: [101, 102, 103, 104, 105]
# under: [73, 74, 75, 76, 77]
# post: [69, 70, 71, 72, 73]

D = len(ddata) 


####### パラメタ初期設定 #######
xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1
x0[-1] = 0.95

####### 観測方程式 ###### # そのリンクにいる確率なので厳密にはダメか？
def mq(max_rssi, rssi, freq):     
#freq): 
# def mq(max_rssi, mean_rssi, dens, rssi, x)
    return (rssi/max_rssi + freq)  # + freq)

####### 観測モデル ###### 
def measuring_model(x):
    qall = np.zeros(len(df_link_odds)*2+2)              # useridとabsorptionの列用
    for file_name in file_list:
    
            if file_name.endswith('.csv'):
                base_name = file_name.split('.')[0]     # 数字部分のみ
                userid = int(base_name)
                    
                file_path = os.path.join(search_folder, file_name)
                dfi = pd.read_csv(file_path) 
                nts = dfi['timestep'].nunique()                

                # 各timestepにおける観測beaconとRSSIを取得し各timestepにおける各リンクの尤度を得る
                grouped = dfi.groupby('timestep')
                dfi_list = [group.reset_index(drop=True) for name, group in grouped]
                    
                qi = np.zeros(len(df_link_odds)*2) 
                for t in range(nts):
                    if t <= nts-2:
                        dfit = dfi_list[t]
                        ts = dfit.loc[0, 'timestep']

                        # 個人i時点tのcountの総和はdfitの長さそのもの
                        count_tot = len(dfit)

                        # tでの観測ビーコンのidとRSSIをタプルにしてリストに入れる（同時刻内の同一ビーコンのデータは，RSSIが大きい方を採用）
                        result = dfit.loc[dfit.groupby('ID')['RSSI'].idxmax()]

                        result_beacon = dfit.groupby('ID')
                        result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]
                        
                        # 各時刻の各linkの観測確率を入れるための配列の用意
                        qit = np.zeros(len(df_link_odds)*2)
                        
                        for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                            dfitj = result_beacon_list[j]
                            count = len(dfitj) # 各ビーコンでの検出回数
                            beaconid = dfitj.loc[0, 'ID']
                            rssi = dfitj['RSSI'].max()
                            slise = dfitj['RSSI']
                            print(f'rssiのデータは{slice}で最大のものは{rssi}')

                            linklist = linklist_near_j[beaconid-1]  # ビーコンから10m圏内のリンクのリスト
                            max_rssi = max_jt[beaconid-1, ts-1]         # tsでのbeaconのmaxのrssi
                            rssiratio = max_rssi / rssi 
                            freq = count / count_tot
                            #print(rssiratio, freq)
                            q = mq(max_rssi, rssi, freq) 
    
                            for link in linklist:                       # linkはlinkidそのまま
                                idx = link-1    
                                qit[idx] = q

                        #print(qit)
                        """
                        id_rssi_tuples = list(zip(result['ID'], result['RSSI']))
                        id_rssi_tuples = sorted(id_rssi_tuples, key=lambda l: l[1]) # rssiの昇順で並べ替え，上書きされるとしたら大きな値になるのでOK


                        print(id_rssi_tuples)
                        # 

                        # 各時刻の各linkの観測確率を入れるための配列の用意
                        qit = np.zeros(len(df_link_odds)*2)

                        # 各ビーコン付近（10m圏内）のリンクに確率を与えるのは，linklist_near_jとmqを使えばソッコーである
                        for i in range(len(id_rssi_tuples)): 
                            # id_rssi_tuplesのi列目が検出されたi番目のビーコンの情報らしい
                            beaconid = id_rssi_tuples[i][0]
                            rssi = id_rssi_tuples[i][1]



                            linklist = linklist_near_j[beaconid-1]      # ビーコンから10m圏内のリンクのリスト
                            max_rssi = max_jt[beaconid-1, ts-1]         # tsでのbeaconのmaxのrssi

                            # このtimestepにおいて，全観測回数count_totでcountをわる
                            #freq = count / count_tot
                            q = mq(max_rssi, rssi) 
                            #, freq)                      
                            # # 帰属確率
                                
                            for link in linklist:                       # linkはlinkidそのまま
                                idx = link-1    
                                qit[idx] = q
                        """
                    # 最後のtimestepでは確定的に一つのリンクに確率1を付与する
                    if t == nts-1:
                        qit = np.zeros(len(df_link_odds)*2)             # dが21の時，最後のリンクは20→21のやつに絞っている，番号は21，idxは20 ハチ公33ならindex32
                        qit[32] = 1

                    # ユーザiが時刻tに各リンクにいた確率
                    qi = np.vstack((qi, qit))

                # 最初の0の列を消す．qi仮完成，観測確率は付与された．続いてこれを向きを考慮して再配分する
                qi = np.delete(qi, 0, axis = 0)

                # 最初と最後のtimestepの観測結果（1次元numpyとして抽出）
                qi_first = qi[0]
                qi_last = qi[nts-1]

                # 最初と最後の階段リンク（staire_linkに該当するインデックスの要素）を0にする（np.arangeの方を変えた．staire_indexは1~の配列）
                qi_first[np.isin(np.arange(len(qi_first))+1, staire_link)] = 0
                qi_last[np.isin(np.arange(len(qi_last))+1, staire_link)] = 0

                # qiの各列で確率の合計が1になるように調整（正規化）
                rowsum = qi.sum(axis=1)
                qi = qi / rowsum[:, np.newaxis] 

                # 最終timestepで最頻(mode)のビーコン→データサンプリングの時，これを満たすようにフィルタリングかける
                dfi_last = dfi_list[nts-1]
                most_common_id = dfi_last['ID'].mode()[0]    

                # 向きを考慮して，最終ビーコンに近い向きを採用
                for j in range(len(qi)):
                    for k in range(1, len(df_link_odds)*2, 2):                      # df_link_integratedを参照してlink kとlink k+1のd_nodeを取得し，最終ビーコンとの距離を比較
                        klink_dnode = df_link_integrated.loc[k-1, 'D']              # Dnodeのnodeidが得られる
                        k1link_dnode = df_link_integrated.loc[k, 'D']               # kと逆方向の偶数リンクのDnodeのnodeid

                        d_k = d_b_node_array[most_common_id - 1, klink_dnode - 1]   # d_b_node_arrayが各nodeとbeaconの距離
                        d_k1 = d_b_node_array[most_common_id- 1, k1link_dnode - 1]
                        if d_k <= d_k1: 
                            continue
                        if d_k > d_k1:                                              # df[k]とdf[k+1]を入れ替える．kはlinkidで今dfはuser_idとtimestepを持つからlink kのindexはk+1(link1はindex2, ,,,)，link k+1はindexk+2
                            qi[j, k] = qi[j, k-1]                                   # k+1にkの観測確率を渡す
                            qi[j, k-1] = 0                                          # kの方は0になる

                # useridの列とabsorptionの列を加える
                userid_array = np.full(nts, userid)
                absorption_array = np.full(nts, most_common_id)
                qi = np.insert(qi, 0, absorption_array, axis=1)                     # absorptionを先頭列に追加
                qi = np.insert(qi, 0, userid_array, axis=1)                         # useridを先頭列に追加
                qall = np.vstack((qall, qi))

    qall = np.delete(qall, 0, axis = 0)
    
    return qall # 全タイムステップ数*userid, absorption, link数（吸収除く）の観測結果


print(measuring_model(0))


start!
rssiのデータは<class 'slice'>で最大のものは-54
rssiのデータは<class 'slice'>で最大のものは-69
rssiのデータは<class 'slice'>で最大のものは-59
rssiのデータは<class 'slice'>で最大のものは-66
rssiのデータは<class 'slice'>で最大のものは-78
rssiのデータは<class 'slice'>で最大のものは-70
rssiのデータは<class 'slice'>で最大のものは-61
rssiのデータは<class 'slice'>で最大のものは-72
rssiのデータは<class 'slice'>で最大のものは-63
rssiのデータは<class 'slice'>で最大のものは-75
rssiのデータは<class 'slice'>で最大のものは-78
rssiのデータは<class 'slice'>で最大のものは-77
rssiのデータは<class 'slice'>で最大のものは-78
rssiのデータは<class 'slice'>で最大のものは-85
rssiのデータは<class 'slice'>で最大のものは-90
rssiのデータは<class 'slice'>で最大のものは-64
rssiのデータは<class 'slice'>で最大のものは-87
rssiのデータは<class 'slice'>で最大のものは-74
rssiのデータは<class 'slice'>で最大のものは-77
rssiのデータは<class 'slice'>で最大のものは-79
rssiのデータは<class 'slice'>で最大のものは-67
rssiのデータは<class 'slice'>で最大のものは-61
rssiのデータは<class 'slice'>で最大のものは-79
rssiのデータは<class 'slice'>で最大のものは-87
rssiのデータは<class 'slice'>で最大のものは-68
rssiのデータは<class 'slice'>で最大のものは-59
rssiのデータは<class 'slice'>で最大のものは-82
rssiのデータは<class 'slice'>で最大のものは-74
rssiのデータは<cla

In [ ]:
##### 同時推定コード #####
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

start_time = time.time()
print('start!')

####### reading data #######
df_quater = pd.read_csv('/home/matsunaga/res2023/data/20230130_08/20230130_0815_kai.csv')

df_link_odds = pd.read_csv("/home/matsunaga/res2023/data/nw/shibuya_link_1112/shibuya_link_post_odds.csv")
l = len(df_link_odds)

df_link_integrated = pd.read_csv('/home/matsunaga/res2023/data/nw/shibuya_link_post_integrated?.csv')
L = len(df_link_integrated)

search_folder = "/home/matsunaga/res2023/data/20230130_08/user_stock_kai0015_end21/groups/group_3"
file_list = list(os.listdir(search_folder))

df_node = pd.read_csv('/home/matsunaga/res2023/data/nw/shibuya_node_corrected.csv')
df_ble = pd.read_csv('/home/matsunaga/res2023/data/nw/ble_nw.csv')

###### 各linkのoとdの座標を入れておく配列 ###### # 高さ方向の検出を排除（床の遮蔽とか考え出すとやばいので）
oddslink_loc_array = [] 
for i in range(len(df_link_odds)):
    linkid = df_link_odds.loc[i, 'linkid']
    O = df_link_odds.loc[i, 'O']
    D = df_link_odds.loc[i, 'D']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] * 1000
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] * 1000 # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### ビーコンとリンク線分の最短距離返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    
    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    
    # 垂直ベクトルのノルムを求める
    distance = np.linalg.norm(v)
    
    # 線分の範囲内であるかチェック
    dot_product = np.dot(v1, segment)
    if dot_product < 0:
        return np.linalg.norm(x - p1)
    elif dot_product > np.dot(segment, segment):
        return np.linalg.norm(x - p2)
    else:
        return distance

###### d_arrayの用意 ###### # 各リンク(奇数側)とbleビーコンとの距離配列
d_array = np.zeros((len(df_link_odds), len(df_ble)))

for i in range(len(df_link_odds)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']*1000 # zの値をめちゃ大きくすることで，階を挟んだ電波捕捉を捨象（階の厳密な高さが不明なのと，床板・天井版による電波遮蔽を考慮するため）．階が同じなら0になるので変な影響はない
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink 2i+1, ビーコンjの距離

###### 各ビーコンからxm圏内のリンクリスト ######
linklist_near_j = []
for j in range(len(df_ble)):
    column_j = d_array[:, j]
    link_list = np.where(column_j <= 10)[0] # 10m以下の場所の指定（インデックスが得られる）  # ここは変えてみる価値あり．12.5m, 15m, 20mにしてみる
    link_list = [idx*2 + 1 for idx in link_list] # link_listの各要素はindexなので，2掛けて1を足せばlinkidになる

    linklist_near_j.append(link_list)

###### ノード間距離の配列 ######
d_node_array = np.zeros((len(df_node), len(df_node))) # d_node_arrayのi行j列がnode i+1とnode j+1の間の距離となるように計算
for i in range(len(df_node)):
    xi = df_node.loc[i, 'x']
    yi = df_node.loc[i, 'y']
    zi = df_node.loc[i, 'floor']
    i_loc = np.array([xi, yi, zi])
    
    for j in range(len(df_node)):
        xj = df_node.loc[j, 'x']
        yj = df_node.loc[j, 'y']
        zj = df_node.loc[j, 'floor']
        j_loc = np.array([xj, yj, zj])

        d_node_array[i, j] = np.linalg.norm(i_loc - j_loc)

###### ビーコンとノード間の距離の配列 ###### # ビーコンi+1とnode j+1間の距離
d_b_node_array = np.zeros((len(df_ble), len(df_node)))
for i in range(len(df_ble)):
    xb = df_ble.loc[i, 'x']
    yb = df_ble.loc[i, 'y']
    zb = df_ble.loc[i, 'floor']
    b_loc = np.array([xb, yb, zb])

    for j in range(len(df_node)):
        xj = df_node.loc[j, 'x']
        yj = df_node.loc[j, 'y']
        zj = df_node.loc[j, 'floor']
        j_loc = np.array([xj, yj, zj])

        d_b_node_array[i, j] = np.linalg.norm(b_loc - j_loc)

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
### 20秒にするのでここやり直す．
dens_jt = np.zeros((23, 60), dtype = int)
max_jt = np.zeros((23, 60), dtype = int)
mean_jt = np.zeros((23, 60), dtype = int)
#count_jt = np.zeros((23, 60), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    # beacon_idとtimestepを得る
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列 ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n)
    for i in range(n):
        O = link_data.loc[i, 'O'] # 当該linkのOnode
        D = link_data.loc[i, 'D'] # Dnode
        for j in range(n):
            if ((link_data.loc[j, 'O'] == O) or (link_data.loc[j, 'O'] == D)) or (link_data.loc[j, 'D'] == O) or (link_data.loc[j, 'D'] == D): 
                I[i, j] = 1
    return(I)

####### リンク接続行列 ####### 
def I_withd(link_data):
    n = len(link_data)
    I = np.eye(n)
    for i in range(n):
        D = link_data.loc[i, 'D']
        for j in range(n):
            if link_data.loc[j, 'O'] == D: 
                I[i, j] = 1
    return(I)

###### 接続行列 ###### 
Ind = I_nod(df_link_odds) 

###### 接続行列 ###### 吸収リンクから吸収リンクへの接続は0に直す
Id = I_withd(df_link_integrated) 
Id[64, 64] = 0

staire_link = df_link_odds[df_link_odds['staire'] == 1]['linkid'].to_list() 

###### 改札の吸収リンク
ddata = [65] # 中央の吸収リンクのみにした．NWの設定とセットで変える
# pre: [101, 102, 103, 104, 105]
# under: [73, 74, 75, 76, 77]
# post: [69, 70, 71, 72, 73]

D = len(ddata) 


####### パラメタ初期設定 #######
xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1.3
x0[-1] = 0.95

####### 観測方程式 ###### # そのリンクにいる確率なので厳密にはダメか？
def mq(max_rssi, rssi, freq): # def mq(max_rssi, mean_rssi, dens, rssi, x):
    # return (rssi/max_rssi + freq)
    return (1/(max_rssi - rssi)*10 + freq)

####### 観測モデル ###### 
def measuring_model(x):
    qall = np.zeros(len(df_link_odds)*2+2)              # useridとabsorptionの列用
    for file_name in file_list:
    
            if file_name.endswith('.csv'):
                base_name = file_name.split('.')[0]     # 数字部分のみ
                userid = int(base_name)
                    
                file_path = os.path.join(search_folder, file_name)
                dfi = pd.read_csv(file_path) 
                nts = dfi['timestep'].nunique()                

                # 各timestepにおける観測beaconとRSSIを取得し各timestepにおける各リンクの尤度を得る
                grouped = dfi.groupby('timestep')
                dfi_list = [group.reset_index(drop=True) for name, group in grouped]
                    
                qi = np.zeros(len(df_link_odds)*2) 
                for t in range(nts):
                    if t <= nts-2:
                        dfit = dfi_list[t]
                        ts = dfit.loc[0, 'timestep']

                        # 個人i時点tのcountの総和はdfitの長さそのもの
                        count_tot = len(dfit)

                        # tでの観測ビーコンのidとRSSIをタプルにしてリストに入れる（同時刻内の同一ビーコンのデータは，RSSIが大きい方を採用）
                        result = dfit.loc[dfit.groupby('ID')['RSSI'].idxmax()]

                        result_beacon = dfit.groupby('ID')
                        result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]
                        
                        # 各時刻の各linkの観測確率を入れるための配列の用意
                        qit = np.zeros(len(df_link_odds)*2)
                        
                        for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                            dfitj = result_beacon_list[j]
                            count = len(dfitj) # 各ビーコンでの検出回数
                            beaconid = dfitj.loc[0, 'ID']
                            rssi = dfitj['RSSI'].max()

                            linklist = linklist_near_j[beaconid-1]  # ビーコンから10m圏内のリンクのリスト
                            max_rssi = max_jt[beaconid-1, ts-1]         # tsでのbeaconのmaxのrssi

                            freq = count / count_tot

                            q = mq(max_rssi, rssi, freq) 
                            # print(q)
                            
                            print(f'user{userid}の時刻{t}のビーコン{link}の確率が{q}，なぜなら')

                            for link in linklist:                       # linkはlinkidそのまま
                                idx = link-1    
                                qit[idx] = q
                        
                    # 最後のtimestepでは確定的に一つのリンクに確率1を付与する
                    if t == nts-1:
                        qit = np.zeros(len(df_link_odds)*2)             # dが21の時，最後のリンクは20→21のやつに絞っている，番号は21，idxは20 ハチ公33ならindex32
                        qit[32] = 1
                    # print(qit)
                    # ユーザiが時刻tに各リンクにいた確率
                    qi = np.vstack((qi, qit))
                    
                # 最初の0の列を消す．qi仮完成，観測確率は付与された．続いてこれを向きを考慮して再配分する
                qi = np.delete(qi, 0, axis = 0)

                # 最初と最後のtimestepの観測結果（1次元numpyとして抽出）
                qi_first = qi[0]
                qi_last = qi[nts-1]

                # 最初と最後の階段リンク（staire_linkに該当するインデックスの要素）を0にする（np.arangeの方を変えた．staire_indexは1~の配列）
                qi_first[np.isin(np.arange(len(qi_first))+1, staire_link)] = 0
                qi_last[np.isin(np.arange(len(qi_last))+1, staire_link)] = 0

                # qiの各列で確率の合計が1になるように調整（正規化）
                rowsum = qi.sum(axis=1)
                
                qi = qi / rowsum[:, np.newaxis] 

                print(qi) # nanになるものがある

                # 最終timestepで最頻(mode)のビーコン→データサンプリングの時，これを満たすようにフィルタリングかける
                dfi_last = dfi_list[nts-1]
                most_common_id = dfi_last['ID'].mode()[0]    

                # 向きを考慮して，最終ビーコンに近い向きを採用
                for j in range(len(qi)):
                    for k in range(1, len(df_link_odds)*2, 2):                      # df_link_integratedを参照してlink kとlink k+1のd_nodeを取得し，最終ビーコンとの距離を比較
                        klink_dnode = df_link_integrated.loc[k-1, 'D']              # Dnodeのnodeidが得られる
                        k1link_dnode = df_link_integrated.loc[k, 'D']               # kと逆方向の偶数リンクのDnodeのnodeid

                        d_k = d_b_node_array[most_common_id - 1, klink_dnode - 1]   # d_b_node_arrayが各nodeとbeaconの距離
                        d_k1 = d_b_node_array[most_common_id- 1, k1link_dnode - 1]
                        if d_k <= d_k1: 
                            continue
                        if d_k > d_k1:                                              # df[k]とdf[k+1]を入れ替える．kはlinkidで今dfはuser_idとtimestepを持つからlink kのindexはk+1(link1はindex2, ,,,)，link k+1はindexk+2
                            qi[j, k] = qi[j, k-1]                                   # k+1にkの観測確率を渡す
                            qi[j, k-1] = 0                                          # kの方は0になる

                # useridの列とabsorptionの列を加える
                userid_array = np.full(nts, userid)
                absorption_array = np.full(nts, most_common_id)
                qi = np.insert(qi, 0, absorption_array, axis=1)                     # absorptionを先頭列に追加
                qi = np.insert(qi, 0, userid_array, axis=1)                         # useridを先頭列に追加
                qall = np.vstack((qall, qi))

    qall = np.delete(qall, 0, axis = 0)
    
    return qall # 全タイムステップ数*userid, absorption, link数（吸収除く）の観測結果

#print(measuring_model(x0))
## とりあえずqallの動作確認をしないと→OK

###### 経路選択モデル ####### 

###### 初期設定 ######
V0 = np.full((L, D), -1) # exp(0)=1よりexp(-1)は小さくなるから?
z0 = np.exp(V0)

###### TSを設定しないといけない ###### #### 一斉に発生すると考えて良さそう→とりあえず時間構造化しないで解く
#TS = 10

V = V0
z = z0
# beta = x0[-1] # dRLなのでbeta推定．xの最後の要素をbetaとする
# beta = 0.99 # dRLなのでbeta推定．xの最後の要素をbetaとする

###### 効用関数 ######
def linkv(x): 
    vinst = np.exp(df_link_integrated['length']/10 * x[0]) #+ df_link_integrated['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    return vinst # 出力形式はseries

###### 即時効用行列 ######
def Mset(x): 
    cost = linkv(x)
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

###### 価値関数 ###### 
def newV(x):
    V = np.zeros((L, D))
    z = np.exp(V)
    
    for d in range(D): 
        d_linkid = ddata[d]
        
        d_index = df_link_integrated[df_link_integrated['linkid'] == d_linkid].index 
        z[d_index, d] = 1                   # 目的地がdの時，dでの期待効用Vd(d)=0から
        M = np.zeros((L, L))
        B = np.zeros((L, 1))
        B[d_index, 0] = 1

        # Mをdごとに更新？
        for k in range(L): 
            for a in range(L): 
                Ika = Id[k, a] 
                if Ika == 1:                # 接続してなければ無関係（0のまま）
                    if a == d_index: 
                        M[k, a] = 1         # 吸収リンクの即時効用はexp(0)=1
                    else:
                        M[k, a] = Mset(x)[k, a]
        dL = 100
        zd = z[:, d].reshape(L, 1) 

        # z求解
        count = 0
        while dL >= 0.01: 
            zdd = zd.copy()
            zd = M @ (zdd ** x[-1]) + B 
            dL = np.sum(np.abs(zdd - zd)) 
            # dL = np.linalg.norm(zdd - zd, axis=0) 
            count += 1

        # z更新
        z[:, d] = zd[:, 0] 
        zd = np.where(zd == 0, 1, zd)  
        one_dim_array = np.log(zd).ravel()   # 1次元配列に
        V[:, d] = one_dim_array 

    return z

###### 対数尤度関数 ###### 
def loglikelihood(x):
    LL = 0

    # 観測モデルとの融合
    print(f'パラメタ{x}で観測モデルを回します')
    qall = measuring_model(x)
    print('観測モデルは回りました')

    # numpyのままではグルーピング不可
    mres = pd.DataFrame(qall, columns=['userid', 'absorption'] + [i for i in range(1, len(df_link_odds)*2+1)])

    # 観測結果をdで分割
    grouped = mres.groupby('absorption')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    

    # loglikelihood内で価値関数更新
    Z = newV(x) 

    for d in range(D):
        d_linkid = ddata[d] 
        d_index = df_link_integrated[df_link_integrated['linkid'] == d_linkid].index
        d_index = d_index.tolist()
        d_index = d_index[0] 

        # 価値関数
        z = Z[:, d].reshape(L, 1) # 価値関数は固定じゃなくてloglikelihood内で逐次更新した方がいい→直したらループが終わらないことは無くなった
        z = z ** x[-1]
        ZD = np.tile(z, (1, L))
        ZD = ZD.T

        # Mをdごとに更新（dループの外で問題はなさそう）
        M = np.zeros((L, L))
        for k in range(L): 
            for a in range(L): 
                Ika = Id[k, a] 
                if Ika == 1:                            # 接続してなければ無関係（0のまま）
                    if a == d_index: 
                        M[k, a] = 1                     # 吸収リンクの即時効用はexp(0)=1（接続してたら）
                    else:
                        M[k, a] = Mset(x)[k, a]

        # 選択確率
        Mz = (M @ z != 0) * (M @ z) + (M @ z == 0)  
        MZ = np.tile(Mz, (1, L))  
        p = (M * ZD) / MZ

        # dを目的地とするユーザのデータを読み込む
        df = df_list[d]
        grouped2 = df.groupby('userid')
        df_list2 = [group for name, group in grouped2] # df_list2の長さが目的地をdとするuserの数に等しい
        df_list2 = [data2.reset_index(drop=True) for data2 in df_list2]
        count = 0

        for i in range(len(df_list2)): 
            df_indivi = df_list2[i]
            data_subset = df_indivi.iloc[:, 2:len(df_link_odds) * 2 + 2]
            qi = data_subset.to_numpy()                     # q[t, k]が，個人の時刻t番目のlink k+1の観測確率に対応
            li = 1                                          # 個人ごとの初期の尤度
            
            for t in range(1, len(qi)):                     # timestep1は前を参照しようがないのでtimestep2から処理
                measured_links = np.where(qi[t] != 0)[0]    # list形式．index
                measured_links = measured_links + 1         # linkid

                # 時刻tの時の観測尤度の初期化
                lt = 0
                # 時刻tでの各観測リンクに対しての操作
                for measured_link in measured_links:                        # measured_linkはlinkid
                    pm = 0
                    measured_column = Id[:, measured_link-1]                # linkid-1なのでlink index．measured_linkをdとするリンクのところが1になってる
                    pre_link_list_index = np.where(measured_column == 1)[0] # measured_linkidに接続するリンクpre_link集合を得る．接続行列Idを参照
                    pre_link_list = pre_link_list_index + 1                 # linkid

                    for pre_link in pre_link_list:
                        pm = p[pre_link-1, measured_link-1] * qi[t-1, pre_link-1]# 経路選択モデル．でも多分ここはpのみのが正しいそうか
                        pm = (pm == 0) + (pm != 0) * pm # logzero回避
                        delta = qi[t, measured_link-1] #* qi[t-1, pre_link-1] # クロネッカーのデルタ（単なる同時確率としてみる）
                        lt += delta * np.log(pm) # pmにクロネッカのデルタをかける．これを全てのatとat-1の組に対して実施
                li += lt  # 時刻に対して和をとる
            LL += li # 個人に対して和をとる
            count += 1
    print(f'パラメタ{x}のもとでLLが回り，link{d}の操作終わり，計算したuserは{count}人でLL={LL}') # LLは負のはず
    return -LL

###### 推定部分 ###### 
dL = 100
n = 0

x_init = np.zeros(2)
bounds = [(-10, 10), (0, 1)] # , (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (0, 1)]

# 注意．LLを返す関数frに対して数値微分を実行
def fr(x): 
    return -loglikelihood(x)

def hessian(x: np.array) -> np.array:
    h = 10 ** -4 # 数値微分用の微小量
    n = len(x)
    res = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            e_i, e_j = np.zeros(n), np.zeros(n)
            e_i[i] = 1
            e_j[j] = 1
            
            res[i][j] = (fr(x + h * e_i + h * e_j)
            - fr(x + h * e_i - h * e_j)
            - fr(x - h * e_i + h * e_j)
            + fr(x - h * e_i - h * e_j)) / (4 * h * h)
    return res

def tval(x: np.array) -> np.array:
    print(f'hesse行列の逆行列{np.linalg.inv(hessian(x))}')            # inv逆行列を計算！
    print(f'各パラメタの分散{-np.diag(np.linalg.inv(hessian(x)))}')        # 対角成分取り出して-1倍
    return x / np.sqrt(-np.diag(np.linalg.inv(hessian(x))))

#num_random_initializations = 100
#for i in range(num_random_initializations):
    # x0をランダムにする
    #x0 = np.random.uniform(low=-10, high=10, size=(2,))  # Modify the range and size accordingly
    #print(f'{i+1}回目の乱数設定，x0は{x0}です')
dL = 100
    ######### 一旦whileループを外して初期値だけ変えて計算
while dL >= 0.01:
        print(f"<<<<<<<<<<<<<<<<now at {n}th loop in while cond>>>>>>>>>>>>>>>>>>")
        n += 1
        x = x0
            # V = V0 
            # z = z0 # 価値関数固定→結局Vはloglikelihood内で作ることにしたので与える必要がない

                # 構造推定part1：対数尤度関数最大化
        res = minimize(loglikelihood, x, method='L-BFGS-B', bounds = bounds) #, options={"maxiter":10,"return_all":True}) 
        
            # 推定値でパラメタxを更新
        x0 = res.x

        print("x0  =", x0)
        print("lnL =", -1*res.fun)

            # 構造推定part2：価値関数更新
            # z0 = newV(x0) 
            # zz0 = (z0 == 0) * 1 + (z0 != 0) * z0 
            # V0 = np.log(zz0)

                # 収束条件の計算（パラメタが収束しているか）
        dL = np.sum(np.abs(x - x0)) 
        print("dL =", dL)
        # print(n)
    #### 本来ここまでwhileループ

### hesse行列自作バージョン

hhh = hessian(x0)
print(hhh)
print(f'繰り返し回数{n}')
#tval = x0 / np.sqrt(np.diag(hhh))
tval = tval(x0)

### BFGS
"""
hhh = res.hess_inv #.todense()
    # print(n)
tval = x0 / np.sqrt(np.diag(hhh))
"""

L0 = -1 * loglikelihood(x_init) # 初期尤度
LL = -1 * res.fun

end_time = time.time()
proc_time = end_time - start_time

    ###### 最終結果の出力 ######
print("計算時間")
print(proc_time)
print("Inputdata")
print(search_folder)
print("結果の表示")
print(res)
    #print("説明変数の表示")
    #print(linkv)
    #print("NFXP")
print("初期尤度 = ", L0)
print("最終尤度 = ", LL)
print("ρ値 = ", (L0 - LL) / L0)
print("修正済ρ値 = ", (L0 - (LL - len(x0))) / L0)
print("パラメータ初期値 = ", x_init)
print("パラメータ推定値 = ", x0)
print("時間割引率 = ", x[-1])
print("t値 = ", tval)

In [2]:
# 例としての k リスト
k = [1, 2, 3, 2, 4, 1, 5]
k_list = [1, 6]
# setを使って重複を除外し、再びリストに変換
k_list = list(set(k))

print("重複を除外した k_list:")
print(k_list)


重複を除外した k_list:
[1, 2, 3, 4, 5]


In [3]:
# 例としてのリスト
original_list = [1, 2, 3]
new_elements = [2, 3, 4, 5]

# original_listにnew_elementsの要素を重複を避けて追加する
original_list.extend(list(set(new_elements) - set(original_list)))

print("重複を避けて要素を追加したリスト:")
print(original_list)



重複を避けて要素を追加したリスト:
[1, 2, 3, 4, 5]


In [4]:
import numpy as np 

a = np.array([[1, 2, 3, 4], [1, 34, 5, 6], [5, 6, 2, 66]])

print(a[-1, 3])

66


In [6]:
import numpy as np

def distance_to_midpoint(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    midpoint = (p1 + p2) / 2  # 線分の中点を計算
    
    # 中点と点 x の距離を求める
    distance = np.linalg.norm(x - midpoint)
    
    return distance

# 使用例:
p1 = [0, 0]
p2 = [4, 0]
x = [0, 0]

result = distance_to_midpoint(p1, p2, x)
print("点と線分の中点との距離:", result)


点と線分の中点との距離: 2.0


In [9]:
import numpy as np 
a = np.array([[1, 2, 3, 4], [1, 34, 5, 6], [5, 6, 2, 66]])

link_cands = a[:, 1]
a0 = np.argmax(link_cands) + 1

print(a0)

2


In [11]:
import numpy as np 
a = np.array([[1, 2, 3, 4], [1, 34, 5, 6], [5, 6, 2, 66]])

b = a[:, 3].min()


print(b)

4


In [2]:
import numpy as np

# 例として作成した2次元のNumpy配列（I）を準備します。
I = np.array([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [1, 0, 0]
])

at = 2  # 取得したい列の番号（at列目）

at_col = I[:, at - 1]  # 指定した列を取得
at_connected = np.where(at_col == 1)[0]   # 値が1になっている行番号を取得（+1をすることで1-indexedにする）

print(at_connected)  # at列目で値が1になっている行番号のリストを表示


[0 2]
